# Building the distributed seismicity model

In [ ]:
# cleaner avoid to restart kernel for each code modification, use it just when alone
from cleaner import modules_cleaner; modules_cleaner()
# load current project if secondary ipynb runned alone
import metys; metys.Metys.secondary()

In [ ]:
%matplotlib inline

import os
import re
import numpy
import scipy
import cPickle as pickle
import matplotlib.pylab as plt

from mbt.tools.area import create_catalogue
from mbt.tools.smooth import Smoothing
from mbt.tools.geo import get_idx_points_inside_polygon
from mbt.sources import OQtSource

from openquake.hazardlib.source import PointSource, SimpleFaultSource
from openquake.hazardlib.mfd.evenly_discretized import EvenlyDiscretizedMFD
from openquake.hazardlib.geo.point import Point
from openquake.hazardlib.geo.geodetic import azimuth, point_at

from openquake.hazardlib.sourcewriter import write_source_model

from hmtk.seismicity.selector import CatalogueSelector

In [ ]:
if len(area_source_ids_list) > 1:
    assert 0 == 1
    
# Load sources
model_key = metys.g_prj.models.current 
data_path = os.path.join(metys.g_prj.folder, '%s_area_sources.pkl' % model_key)
fin = open(data_path,'rb') 
asources = pickle.load(fin)
fin.close()

data_path = os.path.join(metys.g_prj.folder, '%s_fault_sources.pkl' % model_key)
fin = open(data_path,'rb') 
fsources = pickle.load(fin)
fin.close()
    
# Get source data
src = asources[area_source_ids_list[0]]
print 'Processing area source with ID:', area_source_ids_list[0]

In [ ]:
dist_seism_threshold_magnitude = 6.5

def get_xy(line):
    x = []
    y = []
    for pnt in line.points:
        x.append(pnt.longitude)
        y.append(pnt.latitude)
    return x, y 


def get_polygon_from_simple_fault(flt):

    xtrace = []
    ytrace = []
    
    if isinstance(flt, SimpleFaultSource):
        trc = flt.fault_trace
    elif isinstance(flt, OQtSource):
        trc = flt.trace
        
    for pnt in trc:
        xtrace.append(pnt.longitude)
        ytrace.append(pnt.latitude)

    # Get strike direction
    azim = azimuth(xtrace[0], ytrace[0],
                   xtrace[-1], ytrace[-1])
    
    # Compute the dip direction
    dip = flt.dip
    dip_dir = (azim + 90) % 360

    if not hasattr(flt, 'lower_seismogenic_depth'):
        flt.lower_seismogenic_depth = float(metys.g_prj.mod[model_key]['sfs_default_lower_seismogenic_depth'])
     
    if not hasattr(flt, 'upper_seismogenic_depth'):
        flt.upper_seismogenic_depth = float(metys.g_prj.mod[model_key]['sfs_default_upper_seismogenic_depth'])
    
    seism_thickness = flt.lower_seismogenic_depth - flt.upper_seismogenic_depth
    
    # Horizontal distance
    h_dist = seism_thickness / scipy.tan(scipy.radians(dip))
    
    # Compute the bottom trace
    xb = xtrace
    yb = ytrace
    for x, y in zip (xtrace[::-1], ytrace[::-1]):
        nx, ny = point_at(x, y, dip_dir, h_dist)
        xb.append(nx)
        yb.append(ny)
    
    # Create the polygon geometry
    pnt_list = []
    for x, y in zip(xb, yb):
        pnt_list.append((x,y))

    return pnt_list

## Parameters

In [ ]:
area_discretization = 10 # In [km]

## Create the dilated polygon around the area source
NOTE: We don't necessarily need to use the polygon of the area source. In a future version the polygon must be defined in the configuration file or computed automatically.

In [ ]:
new_polygon = src.polygon.dilate(100)
polygon_mesh = new_polygon.discretize(area_discretization)
print 'Number of points: %d' % (len(polygon_mesh))

## Get the earthquakes within the dilated polygon

In [ ]:
# First we get the earthquakes of the catalogue within the dilated polygon 
pickle_filename = os.path.join(metys.g_prj.folder, '%s_catalogue_declustered.pkl' % model_key)
fin = open(pickle_filename, 'rb') 
catalogue = pickle.load(fin)
fin.close()
print 'The catalogue contains %d earthquakes' % (len(catalogue.data['magnitude']))

In [ ]:
# Then we create the subcatalogue for the dilated polygon
cutoff_magnitude = float(metys.g_prj.mod[model_key]['catalogue_cutoff_magnitude'])
fcatal = create_catalogue([src], catalogue, polygon=new_polygon)
selector = CatalogueSelector(catalogue, create_copy=False)
selector.within_magnitude_range(cutoff_magnitude, 10.)

In [ ]:
# Compute scaling factor based on completeness - For the time being we don't consider this.
scalf = numpy.ones((len(fcatal.data['magnitude'])))

## Smoothing 

In [ ]:
smooth = {'gaussian': [50, 25, 0.2],
          'gaussian': [20, 25, 0.8]}
smooth = Smoothing(fcatal, polygon_mesh, 10)

In [ ]:
values = smooth.gaussian(50, 20)

In [ ]:
fig = plt.figure(figsize=(12,8))
plt.scatter(smooth.mesh.lons, smooth.mesh.lats, c=values, vmin=0, vmax=0.4, marker='s', s=15)
plt.plot(src.polygon.lons, src.polygon.lats, 'r')
plt.plot(fcatal.data['longitude'], fcatal.data['latitude'], 'og', mfc='white')

In [ ]:
%%bash
rm tmp*

## Select the nodes of the grid within the area source

In [ ]:
idxp = smooth.get_points_in_polygon(src.polygon)

### Plotting

In [ ]:
fig = plt.figure(figsize=(12,8))
plt.scatter(smooth.mesh.lons[idxp], smooth.mesh.lats[idxp], vmin=0, vmax=0.4, c=values[idxp], marker='s', s=15)
plt.plot(src.polygon.lons, src.polygon.lats, 'r')
for iii, key in enumerate(sorted(src.ids_faults_inside.keys())): 
    tsrc = fsources[key] 
    coord = numpy.array(get_polygon_from_simple_fault(tsrc))
    plt.plot(coord[:,0], coord[:,1], 'r')

## Assigning seismicity to the source
The redistribution of seismicity to the source is done for each cell using as a scaling factor the ratio of the value assigned to the node and the sum of the values of all the nodes within the area source. Note that the mfd assigned to the area source must be an EvenlyDiscretisedMFD instance.

In [ ]:
scaling_factor = 1./sum(values[idxp])
print 'Number of nodes within the area source: %d' % (len(values[idxp]))

print src.mfd.get_annual_occurrence_rates()
rtes = [aa[1] for aa in src.mfd.get_annual_occurrence_rates()]

# mfdpoints is a 2D array with a number of rows equal to the number of points used to 
# represent the area source
mfdpnts = numpy.array([rtes]*len(values))*scaling_factor

print mfdpnts.shape

xxx = numpy.tile(values, (mfdpnts.shape[1], 1)).T
mfdpnts = mfdpnts * xxx

mags = []
for mag, _ in src.mfd.get_annual_occurrence_rates():
    mags.append(mag) 
    
print 'done A'

## Cutting the MFDs of the point sources close to faults

# Create the nrml sources 

In [ ]:
from copy import deepcopy
from openquake.hazardlib.scalerel.wc1994 import WC1994
from openquake.hazardlib.tom import PoissonTOM
from openquake.hazardlib.pmf import PMF
from openquake.hazardlib.geo.nodalplane import NodalPlane 

nrmls = [] 
rupture_mesh_spacing = 2.5
magnitude_scaling_relationship = WC1994()
rupture_aspect_ratio = 2.0
temporal_occurrence_model = PoissonTOM(1.)
nodal_plane_distribution = PMF([(1.0, NodalPlane(0, 90, -90))])
hypocenter_distribution = PMF([(1.0, 7.5)])

for eee, iii in enumerate(idxp):
    jjj = numpy.nonzero(mfdpnts[iii, :] > 0)
    
    if len(list(mfdpnts[iii, jjj][0])) > 0:
        tmfd = EvenlyDiscretizedMFD(src.mfd.min_mag, src.mfd.bin_width, list(mfdpnts[iii, jjj][0]))

        points = PointSource(
            source_id='%s-%d' % (src.source_id, eee), 
            name='', 
            tectonic_region_type=src.tectonic_region_type,
            mfd=tmfd, 
            rupture_mesh_spacing=rupture_mesh_spacing,
            magnitude_scaling_relationship=magnitude_scaling_relationship, 
            rupture_aspect_ratio=rupture_aspect_ratio,
            temporal_occurrence_model=temporal_occurrence_model,
            upper_seismogenic_depth=0.0, 
            lower_seismogenic_depth=src.lower_seismogenic_depth,
            location=Point(smooth.mesh.lons[iii], smooth.mesh.lats[iii]), 
            nodal_plane_distribution=nodal_plane_distribution, 
            hypocenter_distribution=hypocenter_distribution
            )
        nrmls.append(points)

In [ ]:
buff = 2.0
jjj = numpy.nonzero(numpy.array(mags) > 6.5)
chng = numpy.zeros_like((values))
fig = plt.figure(figsize=(12, 10))

if hasattr(src, 'ids_faults_inside'):
    for iii, key in enumerate(sorted(src.ids_faults_inside.keys())): 
        
        # Getting the fault source
        tsrc = fsources[key]
        print 'Source:', key
        
        if 'mfd' in tsrc.__dict__ and tsrc.mfd is not None:
        
            lons, lats = get_xy(tsrc.trace) 

            # Create the polygon representing the surface projection of the fault
            # surface
            coord = numpy.array(get_polygon_from_simple_fault(tsrc))

            min_lon = numpy.min(lons)-buff
            max_lon = numpy.max(lons)+buff
            min_lat = numpy.min(lats)-buff
            max_lat = numpy.max(lats)+buff

            idxs = list(smooth.rtree.intersection((min_lon, min_lat, max_lon, max_lat)))

            iii = get_idx_points_inside_polygon(smooth.mesh.lons[idxs], 
                                                smooth.mesh.lats[idxs],
                                                list(coord[:,0]), 
                                                list(coord[:,1]), 
                                                idxs,
                                                15000.0) 
            
            for tidx in iii:
                plt.plot(smooth.mesh.lons[tidx], smooth.mesh.lats[tidx], 'o')
                mfdpnts[tidx, jjj] = 0.
                chng[tidx] = 1.

plt.plot(src.polygon.lons, src.polygon.lats, 'g', lw=4)
for iii, key in enumerate(sorted(src.ids_faults_inside.keys())): 
        tsrc = fsources[key]
        lons, lats = get_xy(tsrc.trace) 
        coord = numpy.array(get_polygon_from_simple_fault(tsrc))
        plt.plot(coord[:,0], coord[:,1], 'r')

print 'done C'

In [ ]:
from openquake.hazardlib.sourceconverter import SourceGroup
def get_groups(srcl):
    grps = {}
    for src in srcl:
        if src.tectonic_region_type in grps.keys():
            grps[src.tectonic_region_type].append(src)
        else:
            grps[src.tectonic_region_type] = [src]
    return [SourceGroup(key, sources=grps[key]) for key in grps.keys()]

In [ ]:
import operator
from openquake.baselib.general import groupby

# Write the nrml file
model_key = metys.g_prj.models.current 
model_dir = os.path.join(metys.g_prj.folder, 'nrml/%s' % model_key)

if not os.path.exists(model_dir):
    os.makedirs(model_dir)

from openquake.hazardlib.sourcewriter import write_source_model
from openquake.hazardlib.sourceconverter import SourceGroup

sgrps = get_groups(nrmls)

model_name = 'gss_%s.xml' % (src.source_id)
out_model_name = os.path.join(model_dir, model_name)
_ = write_source_model(out_model_name, sgrps, 'Model %s' % (model_key))

print 'Created %s ' % (out_model_name)